In [1]:
import sys 
import pandas as pd
import logging

sys.path.append('../')

import utils.llm 
import utils.scoring

logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

DATA_DIR = '../data/'

TEMPERATURE = 0.7
SAMPLE_N = 5




annotated_df = pd.read_csv(DATA_DIR + 'df_JAQKET.csv')

num_questions = annotated_df.shape[0]

num_questions = 100


In [2]:
llm = utils.llm.LLM()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Loading binary /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No GPU detected! Check your CUDA paths. Proceeding to load CPU-only library...
  warn(msg)


In [3]:
import random


def sampling(query:str,temperature:float,sample_n:int) -> list[str]:
    
    sampled = []
    
    for i in range(sample_n):
        
        # gen random string 
        # rs = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=20))
        
        # q = rs + " " + query
        q = query
        sampled.append(llm.generate(q,temperature))
    
    logger.info("sampled: {}".format(sampled))
    
    return sampled

def get_hallucination_score(query:str,output:str) -> tuple[list[float],list[float],list[float]]: 
    
    sampled = sampling(query,TEMPERATURE,SAMPLE_N)
    pr_scores,re_scores, f1_scores = utils.scoring.get_hallucination_score(output,sampled)
    
    logger.info("hallucination pr_scores: {}".format(pr_scores))
    logger.info("hallucination re_scores: {}".format(re_scores))
    logger.info("hallucination f1_scores: {}".format(f1_scores))
    
    return pr_scores,re_scores, f1_scores




In [4]:
# get_hallucination_score('京都大学の計算機科学コースで行う実験は何ですか','マリオAIを作成する実験です。')

In [5]:
score_df = pd.DataFrame(columns=['qid','pr_score','re_score','f1_score'])



for i in range(num_questions): 

    logger.info("processing {} / {}".format(i,num_questions))
    
    qid = annotated_df.iloc[i]['qid']
    query = annotated_df.iloc[i]['question']
    output = annotated_df.iloc[i]['generated_answer']
    
    print(qid,query,output)
    
    pr_scores,re_scores,f1_scores = get_hallucination_score(query,output)
    
    
    score_df = score_df.append({
            'qid':qid,
            'pr_score':",".join([str(x) for x in pr_scores]), 
            're_score':",".join([str(x) for x in re_scores]), 
            'f1_score':",".join([str(x) for x in f1_scores]),
        },
        ignore_index=True)
    
    



processing 0 / 100
QA20CAPR-0001 和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる動物は何でしょう? マンタレイ
sampled: ['ホッキョククジラ', 'オウムガイ科の一種、クリオネです。', 'マンタ', 'ホッキョクギツネ', 'オウムガイ科 オウムガイ属に属する貝の一種です。貝殻は透明で、真珠層と貝柱があります。オウムガイ科の貝類は世界中に広く分布していますが、日本や韓国の大部分では見られないとされています。']
r_sentences
['マンタレイ']
マンタレイ


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.w

hallucination pr_scores: [0.2779397964477539]
hallucination re_scores: [0.3075794219970703]
hallucination f1_scores: [0.29431746006011966]
processing 1 / 100


/tmp/ipykernel_28939/1749957329.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  score_df = score_df.append({


QA20CAPR-0003 作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう? 黄金の島 イタリア ヴェネツィア マルコ・ポーロは、1453年から1547年までヴェネツィア共和国の大使を務めていました。
ルスティケロは、ヴェネツィアの大使として、マルコ・ポーロから聞いた話をまとめました。
マルコ・ポーロは、ヴェネツィアの大使として、ヴェネツィアの文化や歴史について多くの著作を残しています。
したがって、正解はマルコ・ポーロです。

sampled: ['『ルスティケロの本』。 ルスティケロはイタリアの作家・旅行家でした。彼は17世紀のイタリアに滞在し、多くの文化や人々との出会いから様々な情報を収集しました。この本は、マルコ・ポーロや彼の旅に関する彼の情報と知識を集約したものです。', '『東方見聞録』です。', '『ルスティケロ・デッラ・コンコルディア』。 この作品は、マルコ・ポーロがイタリアから中国に旅した時の模様を、ルスティケロ自身が書き留めた著作です。 この作品は、マルコ・ポーロが自身の旅行記としてではなく、中国に関する著作として出版されたものです。', '『東方見聞録』。この本は、マルコ・ポーロによって書かれたものです。彼は多くの著作を残し、東洋の国々を旅して周りました。彼の旅行記の多くは、イスラム文化やインド帝国などのアジアの国々について語っています。この本は、イタリアからのマルコ・ポーロの帰国に触発されて書かれました。そのため、彼が見たことを記した本として非常に有名です。', '「ルスティケロの書」です。 この作品は、マルコ・ポーロによる旅の物語を、彼の友人であるルスティケロが書き留めたものであり、彼の経験や周囲の状況に合わせたアレンジが施されています。 彼の作品は、マルコ・ポーロとの友情と、彼についての印象的な描写で満ちています。']
r_sentences
['黄金の島 イタリア ヴェネツィア マルコ・ポーロは、1453年から1547年までヴェネツィア共和国の大使を務めていました', 'ルスティケロは、ヴェネツィアの大使として、マルコ・ポーロから聞いた話をまとめました', 'マルコ・ポーロは、ヴェネツィアの大使として、ヴェネツィアの文化や歴史について多くの著作を残しています', 'したがって、正解はマルコ・ポーロです'

...

/tmp/ipykernel_28939/1749957329.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  score_df = score_df.append({


QA20CAPR-0199 「ラティーノ」と呼ばれることもある、スペイン語を母国語とする中南米などからアメリカに移住してきた移民及びその子孫のことを何というでしょう? メキシコ系アメリカ人(メヒコ系アメリカ人)
sampled: ['メキシコ系アメリカ人(Mexican American)', '日系人アメリカ人と呼ばれることもあります。', 'メキシコ系アメリカ人', 'ペルー移民(ペルー・コロンビアなど)、メキシコ移民(メキシコ・キューバなど)、ブラジル移民(ブラジル・ペルーなど)、フィリピン移民(フィリピン・キューバなど)。', '日系人 アメリカに移民した人々は、ハワイやメキシコなど、日本からの移民と共に移住してきた日本人や日系アメリカ人も含まれます。']
r_sentences
['メキシコ系アメリカ人(メヒコ系アメリカ人)']
メキシコ系アメリカ人(メヒコ系アメリカ人)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.w

hallucination pr_scores: [0.21333646774291995]
hallucination re_scores: [0.22132219076156617]
hallucination f1_scores: [0.21850081682205208]


/tmp/ipykernel_28939/1749957329.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  score_df = score_df.append({


In [6]:
logger.info("done") 

done


In [7]:
result_df = pd.merge(annotated_df,score_df,on='qid')


In [8]:
display(result_df)

,qid,question,answer,generated_answer,factuality,faithfulness,collapsed,Unnamed: 7,備考,pr_score,re_score,f1_score
0,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,クリオネ,マンタレイ,0,1,NaN,NaN,NaN,0.2779397964477539,0.3075794219970703,0.29431746006011966
1,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,東方見聞録,黄金の島 イタリア ヴェネツィア マルコ・ポーロは、1453年から1547年までヴェネツィア...,"0,0,0,0","1,1,1,1",NaN,NaN,NaN,"0.34653526544570923,0.26607000827789296,0.2897...","0.2877379298210145,0.23800286054611205,0.24998...","0.32035247087478647,0.2599305272102356,0.27316..."
2,QA20CAPR-0005,『騎士団長殺し』『1Q84』『ノルウェイの森』といった小説の作者は誰でしょう?,村上春樹,村上春樹,1,1,NaN,NaN,NaN,0.033839106559753473,0.08103141784667972,0.06134070158004762
3,QA20CAPR-0007,サッカーで、コーナーキックやフリーキックのようにボールを止めた状態から始めるプレーのことを何...,セットプレー,ゴールキック(goal kick)といいます。\n,0,1,NaN,NaN,NaN,0.22520406246185293,0.2134673714637757,0.22019673585891736
4,QA20CAPR-0009,今から約140億年前に起こったとされる、宇宙の始まりの大爆発のことを何というでしょう?,ビッグバン,ビッグバン」といいます。\n,1,1,NaN,NaN,NaN,0.2448107600212098,0.13099884986877441,0.19189232587814323
...,...,...,...,...,...,...,...,...,...,...,...,...
95,QA20CAPR-0191,1853年から56年までウクライナ地方を中心として起こった、白衣の天使・ナイチンゲールが活躍...,クリミア戦争,クリミア戦争,1,NaN,NaN,NaN,NaN,5.551115123125783e-17,5.551115123125783e-17,5.551115123125783e-17
96,QA20CAPR-0193,JRAでは使える色は4色までと決められている、競馬の騎手がレースで着用する服を何というでしょう?,勝負服 (競馬),ジョッキーベスト,0,NaN,NaN,NaN,NaN,0.13986426591873172,0.159848415851593,0.1500673294067382
97,QA20CAPR-0195,日本人では1974年に佐藤栄作元首相が受賞した、ノーベル賞の中では唯一学問以外のことが対象と...,ノーベル平和賞,経済学メダル,0,NaN,NaN,NaN,NaN,0.235014545917511,0.2689949274063111,0.2529024481773377
98,QA20CAPR-0197,もともとは競馬で1番の馬の場所に棒を立てていたことに由来する、モータースポーツで予選最高位だ...,ポールポジション,ポールポジション(決勝),1,NaN,NaN,NaN,NaN,0.2837721228599549,0.285929787158966,0.2855604767799377


In [9]:
result_df.to_csv(DATA_DIR + 'df_JAQKE_selfcheck.csv',index=False)

In [10]:
result_df = pd.read_csv(DATA_DIR + 'df_JAQKE_selfcheck.csv')

def extract_label_and_scores():
    
    df = result_df.copy()    

    df  = df[['qid','factuality','pr_score','re_score','f1_score']]
    
    label_and_scores = {
        'label':[],
        'pr':[],
        're':[],
        'f1':[]
    }
    
    for i in range(df.shape[0]):
        labels = df.iloc[i]['factuality'].split(',')
        pr_scores = df.iloc[i]['pr_score'].split(',')
        re_scores = df.iloc[i]['re_score'].split(',')
        f1_scores = df.iloc[i]['f1_score'].split(',')
        
        labels = [1 - int(x) for x in labels]
        pr_scores = [float(x) for x in pr_scores]
        re_scores = [float(x) for x in re_scores]
        f1_scores = [float(x) for x in f1_scores]
        
        assert len(labels) == len(pr_scores), f"labels: {labels}, pr_scores: {pr_scores},i: {i}"
        
        label_and_scores['label'].extend(labels)
        label_and_scores['pr'].extend(pr_scores)
        label_and_scores['re'].extend(re_scores)
        label_and_scores['f1'].extend(f1_scores)
        

    return label_and_scores

In [11]:
extract_label_and_scores()

{'label': [1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1],
 'pr': [0.2779397964477539,
  0.34653526544570923,
  0.26607000827789296,
  0.2897722482681274,
  0.24167813062667853,
  0.033839106559753473,
  0.22520406246185293,
  0.2448107600212098,
  0.2747790336608886,
  0.2598912596702576,
  0.05069062709808353,
  0.084438693523407,
  0.2816828608512879,
  0.1725177764892579,
  0.10443842411041254,
  0.24970996379852298,
  0.24017

In [12]:
# AUC scoreを計算   
from sklearn.metrics import roc_auc_score

label_and_scores = extract_label_and_scores()

aucs = {
    'pr':roc_auc_score(label_and_scores['label'],label_and_scores['pr']),
    're':roc_auc_score(label_and_scores['label'],label_and_scores['re']),
    'f1':roc_auc_score(label_and_scores['label'],label_and_scores['f1']),
}

aucs


{'pr': 0.6797385620915033, 're': 0.6315359477124183, 'f1': 0.6633986928104575}

In [13]:
# PR curveを計算
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
%matplotlib notebook

label_and_scores = extract_label_and_scores()

pr_curve = precision_recall_curve(label_and_scores['label'],label_and_scores['pr'])
re_curve = precision_recall_curve(label_and_scores['label'],label_and_scores['re'])
f1_curve = precision_recall_curve(label_and_scores['label'],label_and_scores['f1'])

plt.plot(pr_curve[1],pr_curve[0],label='pr')
plt.plot(re_curve[1],re_curve[0],label='re')
plt.plot(f1_curve[1],f1_curve[0],label='f1')
plt.legend()
plt.xlabel('recall')
plt.ylabel('precision')
plt.show()
plt.savefig('pr_curve.png')

# roc curveを計算
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
%matplotlib notebook

label_and_scores = extract_label_and_scores()

pr_curve = roc_curve(label_and_scores['label'],label_and_scores['pr'])
re_curve = roc_curve(label_and_scores['label'],label_and_scores['re'])
f1_curve = roc_curve(label_and_scores['label'],label_and_scores['f1'])

plt.plot(pr_curve[1],pr_curve[0],label='pr')
plt.plot(re_curve[1],re_curve[0],label='re')
plt.plot(f1_curve[1],f1_curve[0],label='f1')
plt.legend()
plt.xlabel('recall') 
plt.ylabel('precision')
plt.show()

plt.savefig('roc_curve.png')





    



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
# 閾値を適当に設定して、accuracyを計算
th = 0.20


correct_num = {
    'pr':0,
    're':0,
    'f1':0,
}

for i in range(len(label_and_scores['label'])):
    label = label_and_scores['label'][i]
    pr = label_and_scores['pr'][i]
    re = label_and_scores['re'][i]
    f1 = label_and_scores['f1'][i]
    
    if pr > th and label == 1:
        correct_num['pr'] += 1
    elif pr <= th and label == 0:
        correct_num['pr'] += 1
        
    if re > th and label == 1:
        correct_num['re'] += 1
    elif re <= th and label == 0:
        correct_num['re'] += 1
        
    if f1 > th and label == 1:
        correct_num['f1'] += 1
    elif f1 <= th and label == 0:
        correct_num['f1'] += 1


print("acc for th = {}".format(th))
accs = {
    'pr':correct_num['pr'] / len(label_and_scores['label']),
    're':correct_num['re'] / len(label_and_scores['label']),
    'f1':correct_num['f1'] / len(label_and_scores['label']),
}

accs

acc for th = 0.2


{'pr': 0.6585365853658537, 're': 0.6341463414634146, 'f1': 0.6341463414634146}